## 소프트맥스 회귀의 비용 함수를 직접 구현해본다.

### 1. 소프트맥스 회귀의 비용 함수 구현

In [2]:
import torch
import torch.nn.functional as F

In [3]:
torch.manual_seed(1)

#### 1) 파이토치로 소프트맥스의 비용 함수 구현하기 (로우-레벨)

In [4]:
z = torch.FloatTensor([1,2,3])

3개의 원소의 값이 0과 1사이의 값을 가지는 벡터로 변환했음을 알 수 있다.

In [5]:
hypothesis = F.softmax(z, dim=0)
print(hypothesis)

tensor([0.0900, 0.2447, 0.6652])


In [6]:
hypothesis.sum()

tensor(1.)

비용함수를 직접 구현해보자.

임의의 3 x 5 행렬 크기의 텐서를 만든다.

In [7]:
z = torch.rand(3, 5, requires_grad=True)

이제 이 텐서에 대해 소프트맥스 함수를 적용한다.  
단, 각 샘플에 대해서 소프트맥스 함수를 적용하여야 하므로,  
두 번째 차원에 대해서 소프트맥스 함수를 적용한다는 의미로 dim = 1를 전달한다.

In [8]:
hypothesis = F.softmax(z, dim=1)
print(hypothesis)

tensor([[0.2645, 0.1639, 0.1855, 0.2585, 0.1277],
        [0.2430, 0.1624, 0.2322, 0.1930, 0.1694],
        [0.2226, 0.1986, 0.2326, 0.1594, 0.1868]], grad_fn=<SoftmaxBackward0>)


이제 각 행의 원소 합이 1이 되는 텐서로 변환되었다.  
소프트맥스 함수의 출력값은 결국 예측값이다.  
즉, 3개의 샘플에 대해서, 5개의 클래스 중 어떤 클래스가 정답인지 예측한 결과이다.

이제 각 샘플에 대해서 임의의 레이블을 만든다.

In [9]:
y = torch.randint(5, (3,)).long()
print(y)

tensor([0, 2, 1])


이제 각 레이블에 대해서 원-핫 인코딩을 수행한다.

In [ ]:
# 모든 원소가 0의 값을 가진 3 x 5 텐서 생성
y_one_hot = torch.zeros_like(hypothesis)
# 첫번째 파라미터 : dim = 1에 대해서 수행하고,
# 두번째 파라미터(Index tensor) : 위치를 알려줌, 
# Index tensor must have the same number of dimensions as self tensor
# 세번째 파라미터 : 넣을 숫자를 알려줌

# print(y)
# print(y.unsqueeze(1))
# print(y.unsqueeze(1).unsqueeze(1))
# print(y.unsqueeze(1).shape)
# print(y.unsqueeze(1).unsqueeze(1).shape)
y_one_hot.scatter_(1, y.unsqueeze(1), 1)

tensor([0, 2, 1])
tensor([[0],
        [2],
        [1]])
tensor([[[0]],

        [[2]],

        [[1]]])
torch.Size([3, 1])
torch.Size([3, 1, 1])


tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])

In [ ]:
cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()
# print(y_one_hot)
# print(-torch.log(hypothesis))
# print(y_one_hot * -torch.log(hypothesis))
# print((y_one_hot * -torch.log(hypothesis)).sum(dim=1))
# print((y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean())

tensor([[1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.]])
tensor([[1.3301, 1.8084, 1.6846, 1.3530, 2.0584],
        [1.4147, 1.8174, 1.4602, 1.6450, 1.7758],
        [1.5025, 1.6165, 1.4586, 1.8360, 1.6776]], grad_fn=<NegBackward0>)
tensor([[1.3301, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 1.4602, 0.0000, 0.0000],
        [0.0000, 1.6165, 0.0000, 0.0000, 0.0000]], grad_fn=<MulBackward0>)
tensor([1.3301, 1.4602, 1.6165], grad_fn=<SumBackward1>)
tensor(1.4689, grad_fn=<MeanBackward0>)


#### 2) 파이토치로 소프트맥스의 비용 함수 구현하기 (하이-레벨)

파이토치에서는 로그함수와 소프트맥스 함수를 결합한 함수가 존재한다.

In [43]:
# low level
torch.log(F.softmax(z, dim=1))

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]], grad_fn=<LogBackward0>)

In [44]:
# high level
F.log_softmax(z,dim=1)

tensor([[-1.3301, -1.8084, -1.6846, -1.3530, -2.0584],
        [-1.4147, -1.8174, -1.4602, -1.6450, -1.7758],
        [-1.5025, -1.6165, -1.4586, -1.8360, -1.6776]],
       grad_fn=<LogSoftmaxBackward0>)

In [45]:
# low level
(y_one_hot * -torch.log(F.softmax(z,dim=1))).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

In [46]:
# 두 번째 수식
(y_one_hot * - F.log_softmax(z,dim=1)).sum(dim=1).mean()

tensor(1.4689, grad_fn=<MeanBackward0>)

F.nll_loss를 사용하면, 이제 원-핫 벡터를 넣을 필요가 없다.
https://stackoverflow.com/questions/55549843/pytorch-doesnt-support-one-hot-vector

In [47]:
# High level
# 세 번째 수식
F.nll_loss(F.log_softmax(z,dim=1), y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

nll이란 Negative Log Likelihood의 약자이다.  

In [48]:
# 네 번째 수식
F.cross_entropy(z,y)

tensor(1.4689, grad_fn=<NllLossBackward0>)

즉, F.cross_entropy는 비용함수에 소프트맥스 함수까지 포함하고 있다.  
기억해 둬야 구현 시 혼동하지 않는다.

### 2. 소프트맥스 회귀 구현하기

#### 1) 데이터셋 준비

In [49]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [50]:
torch.manual_seed(1)

데이터 분석
- 총 8개의 샘플
- 4개의 feature
- y값을 보니 3개의 class가 있음

In [51]:
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

#### 2) 소프트맥스 회귀 구현하기(로우-레벨)

In [52]:
print(x_train.shape)
print(y_train.shape)

torch.Size([8, 4])
torch.Size([8])


최종 사용할 레이블은 y_train에서 원-핫 인코딩을 한 결과여야 한다.  
클래스의 개수는 3개이므로 y_train에 원-핫 인코딩한 결과는 8 x 3의 개수를 가져야 한다.

In [53]:
y_one_hot = torch.zeros(8,3)
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
print(y_one_hot)

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])


In [54]:
# 모델 초기화
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros((1,3), requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W,b], lr=0.1)

In [55]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # 가설
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1)

    # 비용 함수
    cost = (y_one_hot * -torch.log(hypothesis)).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0 :
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704200
Epoch  200/1000 Cost: 0.623000
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375402
Epoch  800/1000 Cost: 0.329765
Epoch  900/1000 Cost: 0.285072
Epoch 1000/1000 Cost: 0.248155


#### 3) 소프트맥스 회귀 구현하기(하이-레벨)

F.cross_entropy()를 사용하므로, 소프트맥스 함수를 사용할 필요가 없다.

In [56]:
# 모델 초기화
W = torch.zeros((4,3), requires_grad=True)
b = torch.zeros((1,3), requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W,b], lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산
    z = x_train.matmul(W) + b
    cost = F.cross_entropy(z,y_train)

    # Cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.704199
Epoch  200/1000 Cost: 0.622999
Epoch  300/1000 Cost: 0.565717
Epoch  400/1000 Cost: 0.515291
Epoch  500/1000 Cost: 0.467662
Epoch  600/1000 Cost: 0.421278
Epoch  700/1000 Cost: 0.375401
Epoch  800/1000 Cost: 0.329766
Epoch  900/1000 Cost: 0.285073
Epoch 1000/1000 Cost: 0.248155


#### 4) 소프트맥스 회귀 nn.Module로 구현하기

output_dim = 1이였던 선형 회귀와 달리,  
output_dim은 이제 클래스의 개수(3)가 되어야 한다.

In [57]:
# 모델을 선언, 초기화.
# 4개의 특성을 가지고, 3개의 클래스[0,1,2]로 분류
# input_dim = 4, output_dim=3

model = nn.Linear(4,3)

F.cross_entropy()를 사용하므로, 소프트맥스 함수를 사용할 필요가 없다.

In [58]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr = 0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.616785
Epoch  100/1000 Cost: 0.658891
Epoch  200/1000 Cost: 0.573443
Epoch  300/1000 Cost: 0.518151
Epoch  400/1000 Cost: 0.473265
Epoch  500/1000 Cost: 0.433516
Epoch  600/1000 Cost: 0.396563
Epoch  700/1000 Cost: 0.360914
Epoch  800/1000 Cost: 0.325392
Epoch  900/1000 Cost: 0.289178
Epoch 1000/1000 Cost: 0.254148


#### 5) 소프트맥스 회귀 클래스로 구현하기

클래스에 nn.Module을 상속받아, 소프트맥스 회귀를 구현해보자.  

In [ ]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        # super 앞 괄호는 원래, (현재 클래스 이름, self)를 넣었다.
        # Python 3.x 버전부터, 이를 생략해도 동일한 동작을 하게 됐다.

        # super의 괄호를 지우게 되면, super는 객체가 아니라 
        # python 내장 클래스 'super' 자체를 참조하게 된다.
        # TypeError가 발생한다.

        # python 내장 클래스를 참조해서 __init__를 시도하는 건 제한적으로 가능하다.
        # 이를 몽키 패칭이라고 한다.
        # 다른 라이브러리나 코드와 충돌할 수 있고, 유지보수가 어려우므로 권장되지 않는다.
        # 내장 클래스의 디버깅/로깅, 혹은 기능을 확장할 경우 사용할 수 있겠다.
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3

    def forward(self, x):
        return self.linear(x)

모델 인스턴스를 생성하고, SGD 옵티마이저를 설정한다.  
옵티마이저는 모델의 파라미터를 입력받아 학습률 0.1로 경사하강법을 수행한다. 

In [69]:
model = SoftmaxClassifierModel()

# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [72]:
nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train) 

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
    

Epoch    0/1000 Cost: 1.366217
Epoch  100/1000 Cost: 0.722726
Epoch  200/1000 Cost: 0.637564
Epoch  300/1000 Cost: 0.578576
Epoch  400/1000 Cost: 0.527363
Epoch  500/1000 Cost: 0.479315
Epoch  600/1000 Cost: 0.432700
Epoch  700/1000 Cost: 0.386693
Epoch  800/1000 Cost: 0.340930
Epoch  900/1000 Cost: 0.295757
Epoch 1000/1000 Cost: 0.255350
